In [ ]:
Model_name = 'General_Model' # 'Escherichia_coli', 'Salmonella', 'Staphylococcus', 'General_Model', 'Escherichia_coli_General_Model', 'Salmonella_General_Model', 'Staphylococcus_General_Model'

Sourmash_threshold = 0.05

# Initialization

In [ ]:
root_dir = '/content/drive/MyDrive/MRSA datasets'
table_dir = 'Tables/Models Tables/'
table_names = {'Escherichia_coli': 'Escherichia coli_Model_Table.csv',
              'Salmonella' : 'Salmonella enterica_Model_Table.csv',
              'Staphylococcus': 'Staphylococcus aureus_Model_Table.csv',
              'General_Model' : 'General_Model_Table.csv',
              'Escherichia_coli_General_Model': 'Escherichia coli_Species_Model_Table.csv',
              'Salmonella_General_Model' : 'Salmonella enterica_Species_Model_Table.csv',
              'Staphylococcus_General_Model': 'Staphylococcus aureus_Species_Model_Table.csv'}

homology_dir = 'Homology_splitting/'

sourmash_dir = {'Escherichia_coli': homology_dir + 'Sourmash/',
              'Salmonella' : homology_dir + 'Sourmash/',
              'Staphylococcus': homology_dir + 'Sourmash/',
              'General_Model' : homology_dir + 'Sourmash/',
              'Escherichia_coli_General_Model': homology_dir + 'Sourmash/',
              'Salmonella_General_Model' : homology_dir + 'Sourmash/',
              'Staphylococcus_General_Model': homology_dir + 'Sourmash/'}

final_table_dir = table_dir

In [ ]:
!pip install sourmash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.9/332.9 kB 22.5 MB/s eta 0:00:00
  Created wheel for screed: filename=screed-1.1.3-py2.py3-none-any.whl size=96062 sha256=936096bad5030809133014f0c8e2b4a8281ba887e90369c6e514da4a6780821f
  Stored in directory: /root/.cache/pip/wheels/4c/c5/d5/3e3fdf7ca297d976a38274453cb142e83cda0f79c8cc4ac7bb
Successfully built screed


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/MRSA\ datasets

Mounted at /content/drive
/content/drive/MyDrive/MRSA datasets


In [ ]:
import pandas as pd
import networkx as nx
import math
import shlex
import numpy as np

# Sourmash

In [ ]:
table_df = pd.read_csv(table_dir + table_names[Model_name])

# add column for path
table_df['path'] = table_df.apply(lambda sample: 'Datasets/' + sample['Species'] + '/' + sample['Dataset'] + '/' + sample['File Name'], axis = 1)

# save data (fasta) files path
table_df['path'].to_csv(f'{sourmash_dir[Model_name]}{Model_name}_filenames.txt', index=False, header=False)


In [ ]:
len(table_df)

10194

In [ ]:
filenames_file = f'{sourmash_dir[Model_name]}{Model_name}_filenames.txt'
sig_file = f'{sourmash_dir[Model_name]}{Model_name}_genomes.sig'
!sourmash sketch dna -p k=31,scaled=1000 --from-file {shlex.quote(filenames_file)} --output {shlex.quote(sig_file)}

Streaming output truncated to the last 5000 lines.
calculated 1 signatures for 96 sequences in Datasets/Escherichia coli/PATRIC_e.coli/562.23144.fna
... reading sequences from Datasets/Salmonella enterica/PRJNA292666 & PRJNA292661/SRR2566923.fna
calculated 1 signatures for 220 sequences in Datasets/Salmonella enterica/PRJNA292666 & PRJNA292661/SRR2566923.fna
... reading sequences from Datasets/Staphylococcus aureus/PATRIC_genomes/1280.9259.fna
calculated 1 signatures for 68 sequences in Datasets/Staphylococcus aureus/PATRIC_genomes/1280.9259.fna
... reading sequences from Datasets/Salmonella enterica/PRJNA292666 & PRJNA292661/SRR1534879.fna
calculated 1 signatures for 46 sequences in Datasets/Salmonella enterica/PRJNA292666 & PRJNA292661/SRR1534879.fna
... reading sequences from Datasets/Staphylococcus aureus/PATRIC_genomes/1280.30197.fna
calculated 1 signatures for 53 sequences in Datasets/Staphylococcus aureus/PATRIC_genomes/1280.30197.fna
... reading sequences from Datasets/Salmonel

In [ ]:
npy_file = f'{sourmash_dir[Model_name]}{Model_name}_dist.npy'
!sourmash compare {shlex.quote(sig_file)} -o {shlex.quote(npy_file)}

In [ ]:
labels = [l.strip() for l in open(npy_file + '.labels.txt')]
sim = np.load(npy_file)      # Similarity matrix
dist = 1 - sim                 # Convert to distance

edges = [(labels[i], labels[j])
         for i in range(len(labels))
         for j in range(i+1, len(labels))
         if dist[i, j] <= Sourmash_threshold]

G = nx.Graph()
G.add_nodes_from(labels)
G.add_edges_from(edges)
clusters = list(nx.connected_components(G))

In [ ]:
def get_cluster(file_name, clusters):
    for i, cluster in enumerate(clusters):
        if file_name in cluster:
            return i
    return "None"

In [ ]:
table_df['clusters'] = table_df['path'].apply(lambda row: get_cluster(row, clusters))
if 'path' in table_df.columns:
    table_df.drop('path', axis = 1, inplace = True)
table_df.to_csv(f"{final_table_dir}{table_names[Model_name][:-4]}_clusters.csv", index=False, sep=',', header=True, na_rep='N/A')

In [ ]:
print(table_df['clusters'].value_counts())